In [144]:
import sys
sys.path.append( "/home/peer/PycharmProjects/neslearng")

In [145]:
import pandas
from hsa.ba.analyse_ram import ram_from_movie
# from hsa.ba.rewards import time_left, make_delta_potential
import hsa.ba.rewards as rewards
import importlib

In [146]:
importlib.reload(rewards)

<module 'hsa.ba.rewards' from '/home/peer/PycharmProjects/neslearng/hsa/ba/rewards.py'>

In [147]:
%matplotlib

Using matplotlib backend: TkAgg


In [19]:
movies = {}

In [ ]:
movies[5] = pandas.DataFrame(ram_from_movie("../../movies/5_1-1_without-shortcut.fm2"))

In [ ]:
movies[6] = pandas.DataFrame(ram_from_movie("../../movies/6_1-1_with_shortcut.fm2"))

In [ ]:
# /home/peer/PycharmProjects/neslearng/movies/7_1-1_death_and_checkpoint_no_shortcut.fm2
movies[7] = pandas.DataFrame(ram_from_movie("../../movies/7_1-1_death_and_checkpoint_no_shortcut.fm2"))

In [ ]:
# /home/peer/PycharmProjects/neslearng/movies/8_1-1_death_and_shortcut.fm2
movies[8] = pandas.DataFrame(ram_from_movie("../../movies/8_1-1_death_and_shortcut.fm2"))

In [ ]:
# /home/peer/PycharmProjects/neslearng/movies/9_1-1_death_in_shortcut.fm2
movies[9] = pandas.DataFrame(ram_from_movie("../../movies/9_1-1_death_in_shortcut.fm2"))

In [ ]:
# /home/peer/PycharmProjects/neslearng/movies/10_1-1_slow_with_powerup.fm2
movies[10] = pandas.DataFrame(ram_from_movie("../../movies/10_1-1_slow_with_powerup.fm2"))

In [ ]:
movies[11] = pandas.DataFrame(ram_from_movie("../../movies/11_1-1_small_walking.fm2"))

In [ ]:
movies[12] = pandas.DataFrame(ram_from_movie("../../movies/12_1-1_dieing_to_first_gumba.fm2"))

In [ ]:
movies[13] = pandas.DataFrame(ram_from_movie("../../movies/13_1-1_dieing_to_gumbas.fm2"))

In [3]:
movie_paths = !ls ../../movies/*.fm2

In [10]:
[path[13:15] for path in movie_paths]

['10', '11', '12', '13', '5_', '6_', '7_', '8_', '9_']

In [20]:
for movie_path in movie_paths:
    try:
        index = int(movie_path[13:15])
    except: 
        index = int(movie_path[13])
    movies[index] = pandas.DataFrame(ram_from_movie(movie_path))


In [ ]:
delta_potential = rewards.make_delta_potential()

def reward_func(ram):
    return {"delta_potential": delta_potential(ram),
            "time_left": rewards.reward_for_time_left(ram)/10#,
            #"current_screen": ram[0x071A],
            #"playerstate": ram[0x000E]
           }

def rewards_from_frame(df):
    return pandas.DataFrame.from_dict((reward_func(ram) for index, ram in df.iterrows()))

In [ ]:
rewards_from_frame(movies[5]).plot()

In [ ]:
rewards_from_frame(movies[7]).plot()

Note the behavior for checkpoints.
Going down to the start of the level, and then going back up for being teleport forward to the middle

## Main reward function

In [ ]:
def main_reward_series(df):
    main_reward = rewards.make_main_reward()
    return pandas.Series((main_reward(ram) for index, ram in df.iterrows()))

In [ ]:
main_reward_series(movies[6]).plot()

In [ ]:
rewards_from_frame(movies[10])["time_left"].max()

## Fine x Posistion Potential

In [71]:
def fine_x(df):
    x_ready = rewards.with_residual(rewards.fine_x_pos,None)
    return pandas.DataFrame((x_ready(row[1])for row in df.iterrows()))
    #return df.apply(lambda row: x_ready(row)[0],axis=1)

In [52]:
def fine_x_resets(df):
    return df.apply(lambda row: rewards.fine_x_pos(row)[1],axis=1)

In [56]:
def delta_fine_x(df):
    delta_x = rewards.make_delta_fine_x_pos()
    return df.apply(delta_x,axis=1)

In [72]:
fine_x(movies[6]).plot()

In [110]:
delta_fine_x(movies[6]).plot()

## actual fine main reward

In [142]:
def fine_reward_series(df):
    main_reward = rewards.make_fine_potential(kind="delta")
    return pandas.Series((main_reward(ram) for index, ram in df.iterrows()))

In [115]:
def fine_reward_details(df):
    main_reward = rewards.make_fine_potential(kind="all")
    return pandas.DataFrame.from_records((main_reward(ram) for index, ram in df.iterrows()))

In [137]:
fine_reward_series(movies[5]).plot()

In [141]:
fine_reward_series(movies[6])[10:].plot()

In [132]:
fine_reward_details(movies[6])[10:].plot()

In [126]:
movie_6_details = fine_reward_details(movies[6])

In [127]:
movie_6_details.iloc[1225]

coarse                             0
current_coarse_potential           9
current_sum                     9.75
player_state                       2
state                       going up
Name: 1225, dtype: object

In [128]:
movie_6_details.iloc[1231]

coarse                             0
current_coarse_potential           9
current_sum                        9
player_state                       0
state                       going up
Name: 1231, dtype: object

In [134]:
fine_reward_series(movies[12])[40:].plot()

In [135]:
fine_reward_series(movies[13])[40:].plot()

## Finer main reward

In [149]:
def finer_reward_series(df):
    main_reward = rewards.make_finer_potential()
    return pandas.Series((main_reward(ram) for index, ram in df.iterrows()))

In [151]:
finer_reward_series(movies[6])[40:].plot()